### Import Pandas


`Pandas` is a powerful data analysis and manipulation tool, offering easy-to-use data structures and analysis tools for Python

In [2]:
import pandas as pd

### Define the file path

In [3]:
file_path = r"C:\Users\claud\Desktop\data_preparation_using_python\exercise_03_how_to_create_a_calendar_dimension\exercise_03.xlsx"

### Load the Excel File

In [4]:
# Load the Excel file
try:
    with pd.ExcelFile(file_path) as excel_file:

        # Get the sheet names
        sheet_names = excel_file.sheet_names
        print("Sheet Names in the Excel File:" , sheet_names)

        # Create a dictionary to store the DataFrames
        dataframes = {}

        # Iterate over the sheet names and convert each one into a DataFrame
        for sheet in sheet_names:
            dataframes[sheet] = excel_file.parse(sheet)

except FileNotFoundError:
    print (f"Error: The file at path {file_path} was not found")
except ImportError as e:
    print(f"Error {e}")
    print("Please install the missing dependency by running: pip install openpyxl or Pandas")
except Exception as e:
    print(f"Error reading the Excel File {e}")    

Sheet Names in the Excel File: ['dim_customer', 'dim_employee', 'dim_employee_sales_territory', 'dim_geography', 'dim_reseller', 'dim_product', 'dim_sales_territory', 'fact_reseller_sales']


### Merge

In [5]:
dataframes['fact_reseller_sales'].head(1)

,id_order,Order date,Due date,Ship date,id_product,id_reseller,id_employee,id_sales_territory,Order Quantity
0,SO43897,2017-08-25,2017-09-04,2017-09-01,235,312,282,4,2


In [6]:
fact_sales = dataframes['fact_reseller_sales'].copy()

In [12]:
# Convert Order date to datetime
fact_sales['Order date'] = pd.to_datetime(fact_sales['Order date'])

# Find the minimum and maximum date in your DataFrame
min_date = fact_sales['Order date'].min()
max_date = fact_sales['Order date'].max()

# Get the minimum and maximum year
min_year = min_date.year
max_year = max_date.year

# Create a DataFrame for your calendar dimension
calendar = pd.DataFrame(columns=['date'])

# Generate a continuous range between January 1st of the minimum year and December 31st of the maximum year
calendar['date'] = pd.date_range(start=pd.Timestamp(min_year, 1, 1), end=pd.Timestamp(max_year, 12, 31))

# Add additional columns
calendar['year'] = calendar['date'].dt.year
calendar['semester'] = (calendar['date'].dt.month - 1) // 6 + 1
calendar['quarter'] = calendar['date'].dt.quarter
calendar['month'] = calendar['date'].dt.month
calendar['month_name'] = calendar['date'].dt.strftime('%B')
calendar['month_start'] = calendar['date'].dt.to_period('M').dt.start_time

# Calculate last day of each month
calendar['month_end'] = calendar['date'] + pd.offsets.MonthEnd(0)
calendar['day'] = calendar['date'].dt.day
calendar['day_of_week'] = calendar['date'].dt.dayofweek + 1  # Monday is 1, Sunday is 7
calendar['day_name'] = calendar['date'].dt.strftime('%A')
calendar['week_of_year'] = calendar['date'].dt.isocalendar().week

# Calculate week of month
calendar['week_of_month'] = calendar['date'].dt.day // 7 + 1  # Custom logic for week of month

# Export to CSV
calendar.to_csv('calendar_dimension.csv', index=False)

# Display the resulting calendar DataFrame
print(calendar)


           date  year  semester  quarter  month month_name month_start  \
0    2017-01-01  2017         1        1      1    January  2017-01-01   
1    2017-01-02  2017         1        1      1    January  2017-01-01   
2    2017-01-03  2017         1        1      1    January  2017-01-01   
3    2017-01-04  2017         1        1      1    January  2017-01-01   
4    2017-01-05  2017         1        1      1    January  2017-01-01   
...         ...   ...       ...      ...    ...        ...         ...   
1456 2020-12-27  2020         2        4     12   December  2020-12-01   
1457 2020-12-28  2020         2        4     12   December  2020-12-01   
1458 2020-12-29  2020         2        4     12   December  2020-12-01   
1459 2020-12-30  2020         2        4     12   December  2020-12-01   
1460 2020-12-31  2020         2        4     12   December  2020-12-01   

      month_end  day  day_of_week   day_name  week_of_year  week_of_month  
0    2017-01-31    1            7  